In [1]:
from optuna import visualization
from optuna import Trial
import pandas as pd
from lib.utils import load_config
from lib.PCA_Transformer import PCA_Transform
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from optuna import logging

/home/piotr/projects/ml-project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn import set_config
set_config(transform_output="default")

In [3]:
config = load_config()
df = pd.read_csv('processed/apartments.csv', index_col = 0)
target = config['sources']['apartments']['target']
pca_columns = config['sources']['apartments']['pca']['columns']
drop_columns = ['rooms']
to_drop = drop_columns + [target] #pca_columns + 
X = df.drop(to_drop, axis =1)
#X_ohe = pd.get_dummies(X)
y = df[target]
cat_cols = ['city']
num_cols = ['squareMeters','centreDistance','buildYear','latitude','longitude']

In [4]:
cols_pca = ['schoolDistance', 'clinicDistance', 'postOfficeDistance', 'kindergartenDistance','restaurantDistance','collegeDistance','pharmacyDistance']

In [5]:
X.head()

,city,type,squareMeters,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,...,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,ownership,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom
0,szczecin,blockOfFlats,63.00,4.0,10.0,1980.0,53.378933,14.625296,6.53,9.0,...,0.105,1.652,NaN,0.413,condominium,yes,yes,yes,no,yes
1,szczecin,blockOfFlats,36.00,8.0,10.0,NaN,53.442692,14.559690,2.15,16.0,...,0.291,0.348,1.404,0.205,cooperative,no,yes,yes,no,yes
2,szczecin,tenement,73.02,2.0,3.0,NaN,53.452222,14.553333,3.24,9.0,...,0.246,0.300,1.857,0.280,condominium,no,no,no,no,no
3,szczecin,tenement,87.60,2.0,3.0,NaN,53.435100,14.532900,2.27,32.0,...,0.359,0.101,0.310,0.087,condominium,yes,yes,no,no,yes
4,szczecin,blockOfFlats,66.00,1.0,3.0,NaN,53.410278,14.503611,4.07,1.0,...,0.704,0.501,2.138,0.514,condominium,no,no,no,no,no


In [6]:
df.isna().sum().sort_values(ascending=False)/len(df) * 100

floor                   18.185665
buildYear               17.302301
collegeDistance          2.988627
floorCount               1.385877
clinicDistance           0.465485
restaurantDistance       0.306797
pharmacyDistance         0.142819
postOfficeDistance       0.132240
schoolDistance           0.074054
kindergartenDistance     0.068765
hasStorageRoom           0.000000
hasSecurity              0.000000
hasElevator              0.000000
hasBalcony               0.000000
hasParkingSpace          0.000000
ownership                0.000000
city                     0.000000
type                     0.000000
poiCount                 0.000000
centreDistance           0.000000
longitude                0.000000
latitude                 0.000000
rooms                    0.000000
squareMeters             0.000000
price                    0.000000
dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18905 entries, 0 to 18904
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   city                  18905 non-null  object 
 1   type                  18905 non-null  object 
 2   squareMeters          18905 non-null  float64
 3   rooms                 18905 non-null  float64
 4   floor                 15467 non-null  float64
 5   floorCount            18643 non-null  float64
 6   buildYear             15634 non-null  float64
 7   latitude              18905 non-null  float64
 8   longitude             18905 non-null  float64
 9   centreDistance        18905 non-null  float64
 10  poiCount              18905 non-null  float64
 11  schoolDistance        18891 non-null  float64
 12  clinicDistance        18817 non-null  float64
 13  postOfficeDistance    18880 non-null  float64
 14  kindergartenDistance  18892 non-null  float64
 15  restaurantDistance    18

In [8]:
num_columns = list(X.select_dtypes("number").columns.values)
obj_columns = list(X.select_dtypes("object").columns.values)

In [9]:
num_columns

['squareMeters',
 'floor',
 'floorCount',
 'buildYear',
 'latitude',
 'longitude',
 'centreDistance',
 'poiCount',
 'schoolDistance',
 'clinicDistance',
 'postOfficeDistance',
 'kindergartenDistance',
 'restaurantDistance',
 'collegeDistance',
 'pharmacyDistance']

In [10]:
len(num_columns),len(obj_columns), len(df.columns)

(15, 8, 25)

In [11]:
X[X.select_dtypes("object").columns.to_list()]  = X[X.select_dtypes("object").columns.to_list()].astype('category')
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18905 entries, 0 to 18904
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   city                  18905 non-null  category
 1   type                  18905 non-null  category
 2   squareMeters          18905 non-null  float64 
 3   floor                 15467 non-null  float64 
 4   floorCount            18643 non-null  float64 
 5   buildYear             15634 non-null  float64 
 6   latitude              18905 non-null  float64 
 7   longitude             18905 non-null  float64 
 8   centreDistance        18905 non-null  float64 
 9   poiCount              18905 non-null  float64 
 10  schoolDistance        18891 non-null  float64 
 11  clinicDistance        18817 non-null  float64 
 12  postOfficeDistance    18880 non-null  float64 
 13  kindergartenDistance  18892 non-null  float64 
 14  restaurantDistance    18847 non-null  float64 
 15  college

In [12]:
cat_columns = X.select_dtypes("category").columns.to_list()

In [153]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=13)

In [14]:
from lib.optuna.columns import *
from lib.optuna.encoders import *
from lib.optuna.algos import *
from lib.optuna.scalers import *
from lib.optuna.imputers import *

In [15]:
import lib.optuna.encoders as encoders

In [16]:
import lib.PCA_Transformer as pct

In [17]:
import importlib
importlib.reload(pct)

<module 'lib.PCA_Transformer' from '/home/piotr/projects/ml-project/lib/PCA_Transformer.py'>

In [18]:
from sklearn.pipeline import Pipeline


def init_numerical_pipeline(trial : Trial, columns: list[str] = None) -> Pipeline:

  steps = [
    ('imputer',init_imputer(trial)),
    ('scaler', init_robust_scaler(trial)),
    ('pca', pct.PCA_Transform(columns))
  ]

  #print(steps)

  pipeline = Pipeline(steps)
  return pipeline

def init_categorical_pipeline(trial : Trial, columns: list[str] = None) -> Pipeline:
  pipeline = Pipeline([
    ('encoder', encoders.init_encoder(trial)),
    ('scaler', init_robust_scaler(trial)) 
  ])
  return pipeline

In [19]:
from sklearn.compose import ColumnTransformer

def init_processor(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> ColumnTransformer:

  selected_numerical_columns = init_columns(trial, numerical_columns)
  selected_categorical_columns = init_columns(trial, categorical_columns)

  numerical_pipeline = init_numerical_pipeline(trial, selected_numerical_columns)
  categorical_pipeline = init_categorical_pipeline(trial)
  
  

  processor = ColumnTransformer([
   ('numerical_pipeline', numerical_pipeline, selected_numerical_columns),
   ('categorical_pipeline', categorical_pipeline, selected_categorical_columns)
  ])
  return processor

def init_model(trial : Trial, numerical_columns : list[str], categorical_columns : list[str]) -> Pipeline:
  processor = init_processor(
    trial, numerical_columns, categorical_columns
  )
  learner = init_learner(trial)
  
  model = Pipeline([
    ('processor', processor),
    ('learner', learner)
  ])
  return model

In [20]:
logging.set_verbosity(
    logging.WARNING
)  

In [71]:
import pickle

In [164]:
from typing import Optional
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from pandas import DataFrame, Series
import numpy as np

def objective(trial : Trial, X : DataFrame, y : np.ndarray | Series, numerical_columns : Optional[list[str]]=None, categorical_columns : Optional[list[str]]=None, random_state : int=42) -> float:
  
  if not (len(numerical_columns) and len(categorical_columns)):
    raise Exception("Numerical and categorical columns must be provided!")
  n_samples = X.shape[0]
  X_train_sample = X.sample(n_samples, random_state=random_state)
  y_train_sample = y.sample(n_samples, random_state=random_state)
  #print(categorical_columns, X.columns)
  model = init_model(trial, numerical_columns, categorical_columns)
  #print(trial.user_attrs)
  kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
  mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False, needs_proba=False)
  scores = cross_val_score(model, X_train_sample, y_train_sample, scoring=mape_scorer, cv=kf)
  #model_to_save = model['learner']
  trial.set_user_attr(key="best_model", value=pickle.dumps(model).hex())
  return np.min([np.mean(scores), np.median([scores])])

In [105]:
def best_model_callback(study, trial):
    print("C1")
    if study.best_trial.number == trial.number:
        print("C2")
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])
#callbacks=[best_model_callback]

In [165]:
from optuna import create_study

study = create_study(study_name='optimization', direction='maximize', storage="sqlite:///optuna.sqlite3", load_if_exists  = True)

study.optimize(lambda trial: objective(trial, X_train, y_train,numerical_columns=num_columns, categorical_columns=cat_columns), callbacks=[best_model_callback], n_trials=2)

[W 2024-02-22 17:14:38,855] Trial 6 failed with parameters: {'floorCount': False, 'latitude': False, 'buildYear': True, 'centreDistance': False, 'longitude': True, 'squareMeters': True, 'floor': False, 'poiCount': False, 'schoolDistance': True, 'pharmacyDistance': True, 'restaurantDistance': True, 'kindergartenDistance': True, 'clinicDistance': True, 'postOfficeDistance': True, 'collegeDistance': True, 'city': False, 'hasBalcony': False, 'hasSecurity': True, 'type': False, 'hasParkingSpace': True, 'hasElevator': True, 'ownership': False, 'hasStorageRoom': True, 'imputing_method': 'knn', 'indicate_missing': 'yes', 'with_centering': True, 'with_scaling': True, 'encoding_method': 'onehot', 'algorithm': 'linear', 'fit_intercept': False} because of the following error: TypeError('Object of type bytes is not JSON serializable').
Traceback (most recent call last):
  File "/home/piotr/projects/ml-project/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
   

TypeError: Object of type bytes is not JSON serializable

In [156]:
print(f"Best value: {study.best_value} (params: {study.best_params})")

Best value: -0.2500698194868202 (params: {'floorCount': True, 'latitude': True, 'buildYear': False, 'centreDistance': True, 'longitude': False, 'squareMeters': False, 'floor': True, 'poiCount': False, 'schoolDistance': True, 'pharmacyDistance': True, 'restaurantDistance': True, 'kindergartenDistance': True, 'clinicDistance': True, 'postOfficeDistance': True, 'collegeDistance': True, 'city': True, 'hasBalcony': False, 'hasSecurity': False, 'type': True, 'hasParkingSpace': False, 'hasElevator': True, 'ownership': True, 'hasStorageRoom': False, 'imputing_method': 'knn', 'indicate_missing': 'yes', 'with_centering': False, 'with_scaling': False, 'encoding_method': 'onehot', 'algorithm': 'extra_forest', 'n_estimators': 830, 'max_depth': 20, 'max_features': 0.3227371977621448, 'bootstrap': False})


In [157]:
model_bytes = study.user_attrs['best_model']['model']

In [158]:
print(bytes.fromhex(model_bytes))

b'\x80\x04\x95B\t\x00\x00\x00\x00\x00\x00\x8c\x10sklearn.pipeline\x94\x8c\x08Pipeline\x94\x93\x94)\x81\x94}\x94(\x8c\x05steps\x94]\x94(\x8c\tprocessor\x94\x8c#sklearn.compose._column_transformer\x94\x8c\x11ColumnTransformer\x94\x93\x94)\x81\x94}\x94(\x8c\x0ctransformers\x94]\x94(\x8c\x12numerical_pipeline\x94h\x02)\x81\x94}\x94(h\x05]\x94(\x8c\x07imputer\x94\x8c\x13sklearn.impute._knn\x94\x8c\nKNNImputer\x94\x93\x94)\x81\x94}\x94(\x8c\x0emissing_values\x94G\x7f\xf8\x00\x00\x00\x00\x00\x00\x8c\radd_indicator\x94\x89\x8c\x13keep_empty_features\x94\x89\x8c\x0bn_neighbors\x94K\x05\x8c\x07weights\x94\x8c\x07uniform\x94\x8c\x06metric\x94\x8c\rnan_euclidean\x94\x8c\x04copy\x94\x88\x8c\x10_sklearn_version\x94\x8c\x051.3.2\x94ub\x86\x94\x8c\x06scaler\x94\x8c\x1bsklearn.preprocessing._data\x94\x8c\x0cRobustScaler\x94\x93\x94)\x81\x94}\x94(\x8c\x0ewith_centering\x94\x89\x8c\x0cwith_scaling\x94\x89\x8c\x0equantile_range\x94G@9\x00\x00\x00\x00\x00\x00G@R\xc0\x00\x00\x00\x00\x00\x86\x94\x8c\runit_va

In [159]:
model = pickle.loads(bytes.fromhex(model_bytes))

In [161]:
model.fit(X_train,y_train)

Pipeline(steps=[('processor',
                 ColumnTransformer(transformers=[('numerical_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler(with_centering=False,
                                                                                with_scaling=False)),
                                                                  ('pca',
                                                                   PCA_Transform(all_columns=['floorCount',
                                                                                              'latitude',
                                                                                              'centreDistance',
                                                                                              'floor',
                                                                                              'schoolDistance',
                                                                                              'pharmacyDistance',
                                                                                              'restaurantDistance',
                                                                                              'kindergartenDistance...
                                                   'postOfficeDistance',
                                                   'collegeDistance']),
                                                 ('categorical_pipeline',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(sparse_output=False)),
                                                                  ('scaler',
                                                                   RobustScaler(with_centering=False,
                                                                                with_scaling=False))]),
                                                  ['city', 'type',
                                                   'hasElevator',
                                                   'ownership'])])),
                ('learner',
                 ExtraTreesRegressor(max_depth=20,
                                     max_features=0.3227371977621448,
                                     n_estimators=830, n_jobs=-1,
                                     random_state=42))])

In [162]:
y_pred = model.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred)
print(mape)


0.24619915690923752


In [43]:
study_df = study.trials_dataframe()
study_df.sort_values('value', ascending=False).head()

,number,value,datetime_start,datetime_complete,duration,params_algorithm,params_bootstrap,params_buildYear,params_centreDistance,params_city,...,params_pharmacyDistance,params_poiCount,params_postOfficeDistance,params_restaurantDistance,params_schoolDistance,params_squareMeters,params_type,params_with_centering,params_with_scaling,state
28,28,-0.116479,2024-02-20 11:41:10.127704,2024-02-20 11:41:26.622869,0 days 00:00:16.495165,forest,False,True,True,True,...,True,False,True,True,True,True,False,True,False,COMPLETE
33,33,-0.119815,2024-02-20 11:53:12.370758,2024-02-20 11:53:28.131385,0 days 00:00:15.760627,forest,False,True,True,True,...,True,False,True,True,True,True,False,True,False,COMPLETE
32,32,-0.119997,2024-02-20 11:48:07.398265,2024-02-20 11:48:23.952569,0 days 00:00:16.554304,forest,False,True,True,True,...,True,False,True,True,True,True,False,True,False,COMPLETE
31,31,-0.120288,2024-02-20 11:47:46.622718,2024-02-20 11:48:07.373854,0 days 00:00:20.751136,forest,False,True,True,True,...,True,False,True,True,True,True,False,True,False,COMPLETE
35,35,-0.124861,2024-02-20 11:57:00.357053,2024-02-20 11:57:13.977172,0 days 00:00:13.620119,forest,False,True,True,True,...,True,False,True,True,True,True,False,True,False,COMPLETE


In [51]:
params_list = [param for param in study_df.columns if param.startswith("params_")]
print(params_list)

['params_algorithm', 'params_bootstrap', 'params_buildYear', 'params_centreDistance', 'params_city', 'params_clinicDistance', 'params_collegeDistance', 'params_encoding_method', 'params_fit_intercept', 'params_floor', 'params_floorCount', 'params_hasBalcony', 'params_hasElevator', 'params_hasParkingSpace', 'params_hasSecurity', 'params_hasStorageRoom', 'params_imputing_method', 'params_indicate_missing', 'params_kindergartenDistance', 'params_latitude', 'params_longitude', 'params_max_depth', 'params_max_features', 'params_n_estimators', 'params_ownership', 'params_pharmacyDistance', 'params_poiCount', 'params_postOfficeDistance', 'params_restaurantDistance', 'params_schoolDistance', 'params_squareMeters', 'params_type', 'params_with_centering', 'params_with_scaling']


In [55]:
params = study_df.sort_values('value', ascending=False)[params_list][:1].to_dict('records')[0]
params_proper_names = {k.replace("params_",''):v for k,v in params.items()}
print(params_proper_names)

{'algorithm': 'forest', 'bootstrap': False, 'buildYear': True, 'centreDistance': True, 'city': True, 'clinicDistance': True, 'collegeDistance': True, 'encoding_method': 'onehot', 'fit_intercept': nan, 'floor': True, 'floorCount': True, 'hasBalcony': True, 'hasElevator': True, 'hasParkingSpace': False, 'hasSecurity': True, 'hasStorageRoom': False, 'imputing_method': 'simple', 'indicate_missing': 'yes', 'kindergartenDistance': True, 'latitude': True, 'longitude': True, 'max_depth': 12.0, 'max_features': 0.6717325134150298, 'n_estimators': 803.0, 'ownership': True, 'pharmacyDistance': True, 'poiCount': False, 'postOfficeDistance': True, 'restaurantDistance': True, 'schoolDistance': True, 'squareMeters': True, 'type': False, 'with_centering': True, 'with_scaling': False}


In [44]:
from optuna.trial import FixedTrial

In [56]:
objective(FixedTrial(params_proper_names), X_train, y_train,numerical_columns=num_columns, categorical_columns=cat_columns)

-0.11666139169496867

In [57]:
study_fin = study.optimize(objective(FixedTrial(params_proper_names), X_train, y_train,numerical_columns=num_columns, categorical_columns=cat_columns), n_trials=2)

[W 2024-02-21 09:08:43,791] Trial 39 failed with parameters: {} because of the following error: TypeError("'numpy.float64' object is not callable").
Traceback (most recent call last):
  File "/home/piotr/projects/ml-project/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
TypeError: 'numpy.float64' object is not callable
[W 2024-02-21 09:08:43,792] Trial 39 failed with value None.


TypeError: 'numpy.float64' object is not callable